<a href="https://colab.research.google.com/github/sbagency/AI-agents-hacks/blob/main/MTSAIR_Cotype_Nano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB

In [2]:
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.3 MB/s eta 0:00:00


In [5]:
!nohup python3 -m vllm.entrypoints.openai.api_server --dtype=half --model MTSAIR/Cotype-Nano --port 8000  > vllm.out 2>&1 &

In [6]:
!ps -ax | grep vllm

   5711 ?        Sl     0:08 python3 -m vllm.entrypoints.openai.api_server --dtype=half --model MTSA
   5772 ?        Rl     0:06 /usr/bin/python3 -m vllm.model_executor.models.registry
   5804 ?        S      0:00 /bin/bash -c ps -ax | grep vllm
   5806 ?        S      0:00 grep vllm


In [9]:
!cat vllm.out

2024-11-26 12:08:26.536951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-26 12:08:26.563134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-26 12:08:26.575008: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 12:08:26.598062: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 12:08:28.245228: W tensorflow/comp

In [15]:
import openai
import pandas as pd
from tqdm import tqdm


model = 'MTSAIR/Cotype-Nano'

client = openai.OpenAI(
    base_url = 'http://localhost:8000/v1',
    api_key='xxx', # required, but unused
)


# Possible system prompt:
# {"role": "system", "content": "Ты — ИИ-помощник. Тебе дано задание: необходимо сгенерировать подробный и развернутый ответ."},
#    messages=[
#          {"role": "user", "content": "Как мне обучить модель meta-llama/Llama-3.2-1B с помощью библиотеки transformers?"}
#        ]
#    )

temperature=0.4 # 0.0 is also allowed
frequency_penalty=0.0
max_tokens=2048
top_p=0.8 # 0.1 is also allowed


#import time
#from openai import OpenAI
#from google.colab import userdata


def llm(messages):
  print(f"{model},temperature={temperature},top_p={top_p},max_tokens={max_tokens}")
  completion = client.chat.completions.create(
  model=model,
  messages=messages,
  temperature=temperature,
  frequency_penalty=frequency_penalty,
  top_p=top_p,
  max_tokens=max_tokens,
  stream=False
  )
  if not hasattr(completion, 'choices'):
    print(completion)
    raise Exception(completion)

  return completion.choices[0].message.content


def llmp(prompt):
  messages=[{"role": "user","content": prompt}]
  return llm(messages)

In [16]:
resp=llmp("Расскажи анекдот про лошадь")
print(resp)

MTSAIR/Cotype-Nano,temperature=0.4,top_p=0.8,max_tokens=2048
Конечно, вот один из анекдотов про лошадь:

---

Однажды лошадь спрашивает другую лошадь:

— Ты не боишься, что я могу тебя убить?

Вторая лошадь отвечает:

— Нет, боюсь, что ты не сможешь меня убить.

---

Надеюсь, этот анекдот вам понравился!


In [17]:
#https://www.youtube.com/watch?v=zr5BLuSgOHo

bigtext="""
привет У меня сегодня в гостях Никита зелинский Никита работает чиф Дета
саентистом в МТС И в целом прошёл очень интересный карьерный путь Никит Привет
Здравствуйте коллеги Ну расскажи немножко у тебя очень крутой опыт рабо в
больших компания какие есть там топ три наверное плюса работы в большой
Топ-3 плюса и минуса работы в большой компании
корпорации и топ три минуса Ну слушай в большой компании У тебя есть все шансы получить зарплату плюс-минус вовремя то
есть в одной из маленьких компаний у нас был такой случай в четырнадцатом году когда 1 апреля всех выстроили вот по
линеечка там 9 утра собрание собственники приехали Говорят это не первоапрельская шутка увальнем каждого
второго по списку чтобы без обид А это было прямо сокращение Ну мы там все там в серую
работали с официальным окладом по зарплате кидались Ну примерно в феврале следующего года Нет всё до
копейки отдали Вот Но короче вот плюсы больших компаний Это вот Ну хорошо
присутствует это один плюс ещё Слушай можно Если
повезёт встретить Ну кого-то кто шарит Угу и получиться потому что в больших
компаниях всегда есть конкуренция там обычно там 10 отделов делают 20 штук которые абсолютно друг друга дублируют
кто-то да сделает нормально быть с первого приседа периодически туда там
попадают нормальные Ребята с классным опытом по разным причинам для многих это промежуточный шаг кто-то наоборот на
пенсию Вот поэтому ты в принципе можешь найти у кого-то получиться ментора какого-то посмотреть там ну я бы это не
формулировать с чуваком пивка попить чтобы узнать как жизнь устроена и что вообще вообще стоит делать что не стоит
Ну в целом можно в маленькой компании Ну на собеседование приходят ребята я там
вотже случай был говорю А что ты делаешь Мы тут скоринговый Модели там значит попросили говорю А как ты Ну короче вот
фотографию загоняю короче врек угу я говорю А как ты это делаешь Ну вот там
же пиксели я говорю А может ты как-то там свёрточные говорит Да нет Так так
нормально там доки ды этот А я говорю А что доки дый Ну мы не мерили я такой А А
почему ты вообще решил так делать в статье написано что за статья то вообще да я почитаю Ну вот вот такие приколы
они там достаточно ча то есть люди изобретают какие-то вобще космические велосипеды не знаю базовых понятий думаю
так и надо у них там этот прям ну там мягко говоря совсем не демократия то есть там по свистку все приходят на рабо
они ещё нанимают себе людей если Компани стт которые смотрят в рот и верят что
это светило да потом они сталкиваются Да нет такое бывает и в крупных компаниях недавно заседал в одном программном
комитете конференции и там тоже дошло до докладов нам человек из компании говорит
не вот эти доклады ни в коем случае не берите это какая-то группа отщепенцы
и вот они вообще ничего не знают в этой теме короче вот моя группа занимается этой темой А те вот не очень ну там
правда к докладам были вопросики мы там отклонили там ряд докладов Вот Но это было Забавно вот поэтому Ну в целом Да
наверное второй плюс - это то что можно найти у кого-то у кого-то получиться посмотреть как надо у тебя огромный шанс
попасть на какой-то leg Как не надо это кстати тоже плюс посмотреть там задёшево как не надо не меняя запись трудовой Вот
но ещё есть маза всё-таки найти какую-то сильную команду к ней прибить подрасти если люди там будут заинтересованы что
там есть Ну я думаю единственная причина почему люди могут быть заинтересованы чтобы ты там оказался 24 на 7 готов
работать Ну в целом волосы не просто так выпадают и виски
Сиде за что-то мы платим за что-то расплачиваемся Ну а минусы ты считаешь
что у работы в большие компаниях есть мес А по сравнению с чем собственно по сравнению с тем что получить миллиард
долларов на свой стартап уехать в Калифорнию я да это правда я не думаю что в больших компаниях есть правда
минусы У тебя есть там некоторые риски и опасности но минусов В текущей
действительности по сравнению с работой в каком-то стартапе или с работой в какой-нибудь маленькой компании
специализированные Ну могут быть только по сравнению с работой в какой-нибудь крутой нбе где у наушника там связи за
рубежом где там прям всё налажено чётенько там интеграция с какими-то передовыми там компаниями на рынке там А
технологическими ну ты попробуй туда попади вот поэтому Ну исходя из базовых
свой которые были у меня наверное это было самое верное решение которое пришёл через череду прямо вот неверных Ну
слушай это на самом деле очень здорово что ты так воспринимаешь работу в больших компаниях Потому что ты в моей
жизни наверное первый человек Кто заявил что вот минусов нет особенно если
принять во внимание Какие минусы в других местах Так что мы с тобой об этом ещё много поговорим в этом интервью Ну
это же эффект базы понятно что если ты там закончил фех с красным дипломом правильного наушника и так далее или
вышку там там там у тебя спе гаранту стажировки и так далее куча грантов и
там материально ты независим конечно ты можешь сразу готовиться там не знаю к
там условно или Google там вот если ты парень попроще да е с НЕФ
получить там профильный опыт Да ещё в крупное компании есть возможности роста по-моему такой практически
безальтернативные шаг Расскажи нам Кто такой Никита зелинский как у него вообще
Кто такой Никита Зелинский?
сформировался такой трезвый взгляд на жизнь который ты сейчас
транслирует уместить Так этот минут я не обещал что будет легко так точно Слушай
ну сейчас я твоими Кстати стараниями работаю в МТС Вот сидим и за ль
платформы и за Вот это всякое и за н кусочками что-то несу свет ль в компанию
МТС А вот до этого я работал в Сбере 6 лет приходил я туда внезапно для себя
абсолютно это выяснилось уже после трудоустройства it аналитиком это Это был конечно Эпик когда-то типа собесе у
тебя вопрос по ходу по emil ты такой О сейчас модельки буду строить выходишь Какой emil Вот вот тебе ноутбук такой
вот знаешь как дрова Примерно вот Пиши
документацию Да ну всякое было Вот до сбера Я работал в департаменте it Москвы
Ну а в Сбере собственно до CD средний плюс дорос то есть в корпоративном блоке
корпоративно ионном есть там департамент который занимается продажами м средний и крупный Вот таких сегментов вот я там
был cdo то есть и за отчётность отвечал за модели и за Фин эффекты от этих моделей там короче за всё за что это
сколько человек тебе нужно было хороводи меня было 46-47 вот так вот менялся вот
начинал Ну типа аналитиком в до сбера Я работал в Московском дете там отдельная история там создали центр анализа данных
такой там с рынка набрали там через такой ну интересный конкурс там
получилось 1500 резюме на 30 мест Вот бы в несколько этапов тасе причём первый
был IQ тест и это был мой шанс тоже после IQ теста из полутысячи осталось 72
которые на команды дальше разбивали бизнес играли Только потом уже собеседование поэтому шансов меня
зарезать было сильно меньше чем обычно да это ну собственно это то место где я впервые столкнулся с этим вселенским
злом который называются hrbp и прочее это сейчас намёк был не
просто этого Ну до этого чуть-чуть в Яндек Спасибо людям которые на путь истины направили Вот Ну а до этого я
занимался чем придётся Вот чем придётся тем и занимался то есть а вот ты
кандидат в имат наук но при этом у тебя образование геологическое ты геологический факультет
Закан В какой момент случился разворот в сторону математики Слушай я поступал на
геофизик Требования по математике такие были достаточно сильные э Почему Я
поступил на геофак потому что на мехмат бы я не прошёл а с таким же набором экзаменов был только геологический
математика письменна устно и русский тогда же не было всех этих ЕГЭ и так далее математика там четыре курса то
есть там всякие там туры там учп там оптимизация там всякие вычислительные
методы вариационное исчисление отдельным курсом всякие функционалы то что у вас фун коном называется там кусочек Фуна
ционного счисления и курсе на Ну как бы обработка сигналов Я занялся курсе на
втором на кафедре курсе к четвёртому докатился что вот есть оказывается такие нейронные сети Кто бы мне про них
рассказал они там как чёрный ящик что-нибудь закинул такое многомерное они что-нибудь там сделали прикольное вот а
я штукой много на матлабе экспериментировал ну пытался там какие-то статей писать дипломы и так далее вот а в какой-то момент по поводу
кандидатского Ну я просто взялся за голову и числился в аспирантуре чтобы армию не забрали и понял что надо с
военкоматом решать капитально да изза одной какой хоть какой-то математическое
образование тире корочку получить о за полгода защитился тело-то нехитрое А вот
ты в тот момент задумался о том чтобы прийти в Science Или это как-то по-другому проо я не знал что такая
отрасль есть Ну а как попал
поле занимался обработкой сигналов там уже что-то Да уже там в Яндек
уже узна что есть таком дил на си дил всякие процедуры
обработки там одноканальной а сигналы всякие деконволюция фильтрации это ну
вообще на самом деле рядом достаточно в прогнозировании временных рядов мне так стать потом Здорово помогало такой опыт
вот а потом Да было Вот это объявление набираем аналитиков данных и мне понравилось слушай там Прикольная
история которые можно найти глазами Ну то есть у людей модель не работает Ты им показываешь Ну ребят Вот в этой точке
две базы слили вот смотрите график числа айди ников от даты такой излом Ну что вы там строите та сначала Посмотрите
глазками и таких фагов было много наверное самое большое что мне нравится в моей работе это фандинг вот если ты
помнишь когда я пришёл в МТС Там как бы заняла там 2 дня чтобы отбить свою зарплату пришёл в команду которая что-то
какие-то скоринг делать что-то ди модель А там же в скоринг как единичка очень важно количество Ну то есть ты реально у
тебя от количества едини Ну то есть дефолтов зависит от того насколько крутая модель нолито мало по определению
не благотворительный фонд для зрителей сделаю ремарку что речь идт про задачу кредитного скоринга когда оценивается
вероятность просрочено платежа по кредиту перед тем как его выдавать так к сожалению получилось что банки такие
корыстные организации и хотят чтобы им платили по кредитам и
вот очень важно а вообще сколько у нас примеров этих просрочек потому что на нихто надо обу представ сера ко
там больше 2 лет сидит там строит эти модельки каждый квартал себе берёт
какую-то цель там там на пол пункта Джини её сделать получше Вот я прихожу
смотрю Ребята а что у вас как бы таргеты не взя они как так не все Ну вот мы с такой табличкой берём Я говорю Ну если у
вас тут поле телефон не заполнено но есть ссылка на табличку с клиентами где поле телефон есть ничего Так вам не
мешает там подставить оного в другое вот табличка с договорами Если вдруг в договоре не указан телефон то он в
карточке клиента указан Да ну так можно было в 3 с поно раза число дефолтов выросло на исторических данных ну
моделька сразу стану там очень-очень сильно ну как бы это фандинг Это пример
фандинг это такой блин тут команда целая мучилась там несколько человек не один год А ты как бы за 2 дня нашёл вот это
прикольно такие моменты в своей профессии Я люблю Ну получается у тебя была достаточно хорошая математическая
подготовка всё-таки и вот что ты можешь сказать для наших слушателей насколько
вообще важно иметь математическое образование Ну в Science Ну тут вопрос
что считать математическим образованием я знавал ребят которые там учились не на мехмате но посещали нму ну которые
открытые Вот И там их какую-то хитрую алгебру
затирания чёрного двадцатилетия мехмата когда по разным причинам связанным с
национальностью не принимали людей и работать и учиться мехмате и вот
возникло такое заведение образовательное где математики учат
наверное даже пожёстче чем на мехмате и там очень много крутых профессоров Да но
только это очень специфические курсы специфические семинары которые предполагаешь ты уже там очень в теме
разл Да И вот я наблюдал вот эти неуспешные попытки получить образование там потому что ребята базы не знали
поэтому Ну хорошо отвечу на твой вопрос Я недавно
проводил собеседование студентов кандидатов в магистратуру прямо топового Московского
ВУЗа и начал с тех пор задавать этот же вопрос на собеседование на Да коллеги не
отвечают Что такое производное Угу Ну в лучшем случае пытаются рассказать Ну вот как бы касательное я говорю а третья
производная - это это уже какойто Микеланджело какие-то фигуры как бы я не
понимаю Вот ну то есть математическая база нужна но она нужна на уровне девятого класса Ну ну хорошо десятого Ну
производно Возможно даже в одиннадцатом проходит Ну не суть сейчас и учится кстати 11 лет Да
никак не могу привыкнуть да то есть доучился сразу в армию такое Ты работал
в Сбере работаешь сейчас в МТС до этого можно сказать что в каком-то смысле
поработал в Яндексе Роме того работал в те Москвы то
Сбер, МТС, Яндекс.Терра (Сейсмотек), ДИТ Москвы - работа в больших компаниях
есть в целом у тебя очень такой большой набор разных организаций это из профильного да то есть вот Ну да
расскажи как они отличались между собой ну то есть вот есть ли различия между там дитом допустим и сбером или там
сбером МТС слуша вот плюсы больших компаний Не устаю говорить о плюсах пото
что они супер неоднородные там в большой компании Ты можешь найти очень крутую команду в которой будет минимум
бюрократии крутые эксперты а ценность на результат высокая продукци культуры культура разработки и можешь найти
абсолютную противоположность вот в маленькой компании у тебя нет такой возможности в большой компании ты там
можешь расти в любую сторону там можешь менять профессию мне предлагали переметнулся
[музыка]
[музыка] вариантов масса не меняя места То есть ты
минимизирует самовар там электровеник сегодня один товарищ сказал То есть ты отвечать за всё делать всё но зарплата у
тебя будет как у эникейщика кстати тоже пару лет работал Окей Ну а всё-таки ты упомянул
бюрократию вообще насколько компании покра отличаются меду собой
большие Т вопрос твого жения пря хочешь сделать те никакая бюрократия Нико в
жизни не помешает Ну только возможно это будет очень долго Ну вопрос как ты это
обос ты покажешь ты заработаешь два ерда тебе надо десятку Ну Твой руководитель с кармана десятку достанет то есть ну как
бы ну ради такого Это просто бонусом больше получит Вот а если да ты хочешь что-то там такое поменять там не особо
материальное чтобы типа заявок поменьше согласовывать и так далее маршруты попроще Ну тут Да тут будешь мучиться
вот никогда не забуду сбе жу кластером модели Да 50 человек мне звонит человек который говорит Вот вы 5 лет назад
заявку оформили на согласование мы её закрыть не можем Закройте причём в приказном порядке там там типа я говорю
тот человек даже не работает на кого она заведена тогда Оформите вот там-то там там чтобы маршрут Пере согласовал
Вот я ему подсказал дорогу и ничего дальше не было со мной по повороти тоже
такой момен я
Вот и смотрел как мой фитор отработал пря какие-то пограничные кейсы там где он был неуверен
там такой смотрю Какая странная проводка звоню в расчётный Центр на Урал говорю
Девчонки тут както ну поясните экономический смысл Вот они такие сейчас
берём в работу с утра прихожу на почте
письмо 200 млн руб и девчонки по звонку какого-то ноунейма Ну с пше
исполнительный директор из там центра развития компетенции дата S или что-то
такое такие скатили Причём я не говорил что там неправильно я хотел узнать почему такая
пара счетов вот нуну понятно На следующее утро разрулил потому что ну как бы вот перед клиентом Я бы
объясняться не захотел Ну прикинь у тебя со счёта 200 лямов там просто пропадает какой-то там аналитик да глубоко копал
такой опапа А что там такое ну то есть это вот да большая организация Слушай ну это как с одной
стороны смешно с другой конечно очень круто Ну это не про бюрократию это про цветовую фнц штанов да то есть пото что
часто там внутри коридоров там называют что вот типа получил новый грейд там типа погоны и часто вот такие вот люди
особенно там старшего поколения особенно в департаментах безопасности они такие военизированные темами общаются Вот Но
это достаточно Забавно выглядит это конечно да такие как вобще вот этот вот мир ролевых игр корпоративных устроен то
есть вот есть у тебя разные грейды есть у тебя там разные зарплаты бонусы эти бонусы наверное на что-то тратят люди
типа там купить с бонусом машину или там вот такие вот слушай штуки есть которые
вот ну все верят что в корпорациях есть вот какой-то такой свой флр Вот расскажи
о нём немного если он есть Или скажи что нет Слушай ну периодически Да ты зае
какому-то другу вот показывает вот типа вот такую штуку там по наме взял с бонуса вот я просто так не умею тратить
бонус Я наоборот вот я сюда шёл там от метро Курской там знаешь если очка идти будет скверик знаешь Чем скверик
известен там под вечером очередь стоит очередь к пустой середине сквери А
почему туда приезжают бесплатный суп раздают поэтому я как-то особо в плане
таких больших трат нет вот ну бы инвестиции портфели туда-сюда Мали Какая
жизне ситуация возник не дай бог кто-то заболеет и такое было поэтому Ну тут я не могу сказать по поводу кичиться
бонусами не кичится Вот Но вот именно название должности там люди конечно Яри
То есть даже не Грейт играет роль А называтся по шека пше Да чтобы Люд было побольше тоже У меня другой товарищ
такой вот смотри у меня 270 человек я говорю Ну у тебя же центр стажировка туда те всех стаж в банка просто туда
оформляет говорит Да ну кто снаружи ты знает представляет вот Андрей там
руководите центром на 270 человек Ну сразу уровень вот не парень-то Вообще
классный но вот это просто так работает в корпорациях да чем больше у тебя поляна Вот это называется тем больше у
тебя там Люд тем ты значишь класне вот а поляну в людях меряют или в Милли кстати
в том числе не в миллиардах в этом плане очень плохо то есть а в русских
компаниях крупных насколько я знаю вот эта вот история сю shing она даже если топами одобряется да типа вот в Сбере
одно время было Давайте до саентистом Да Сейчас расскажу Давайте до саентистом тить роялти от модели Ну пусть он
получает 2% от модели которая в пром поставил потому что сир то есть Кост ийо
у дантистов очень-очень маленький это значи если о наоборот потеряет деньги компании это проблема валидатором нет
потому что 2% у него учтет по жизне не я ещё раз говорю вот то что ты говоришь называется модельный риск и в банках
модельный риск закрывает управление валидации То есть перед тем как ставить модель про они проверяют Окей Вот и как
бы все вопросы к ним так вот да было там про 2% и сначала эти 2% что-то срезали
там что-то жирно будет Давайте полпроцента давать потом ещё что-то в итоге это выразилось что руководитель по
своему усмотрению Может чуть-чуть там премию увеличить Ну то есть ты можешь сделать
модель на миллиард там типа дополнительной чистой прибыли Да ну типа там полтинник за это получить бонус от
руководителя погода ну как бы я считаю это вотре шеринг Это если ты что-то делал ту сделал классное Да чтобы ты
полк миллионов или тысяч не 50.000 руб да дада да ре шеринг Это значит ты
что-то сделал классное и получил какую-то Ну там пропорциональную долю Ну там хотя бы перевыполнил план уже там А
вот и вот что-то компании на это не особо идут потому что не доверяют это такая ну двойственности и про бюрократию
то есть когда ты защищаешь Фин эффект или там перед аудиторами или перед финансиста тебе до каждой копейки там
вот асну я вот ат Вот это всё Ты такой а потом когда Ну смотрит Ну что-то для
такой маленькой команды что-то слишком больший эффект не дадим вам премию денег и так далее достаточно за Ну кстати
полтинник В смысле 50.000 это наверное трудяга а его менеджер может и 50 млн
получить нет нет вот нет ну то есть там совсем менеджер менеджером которые к дам
Data Science не относится Ну может какой каких-то тив дадут 50 лямов там на несколько лет Рассчитано но опять же не
за счёт отти То есть тут как бы двойственной стороны вот Data Science с одной стороны ВС
понимает что нужен Data Science Модели там Фин эффекты с них считают и так далее а с другой стороны менеджеры Ну
они такие Блин ну всё равно же продал конкретный продажник или всё равно вот эта кнопочка на сайте оплатить Вот она
продаёт модельки какие-то хоть ты обложить А тестами что вот без этого
работает плохо что без это у тебя выручка сразу упадёт Ну вот недоверие такое не готова они признавать что вот
такой большой им что насчёт грейдов и вот названий ты говоришь даже название
важно Ну там есть исполнительный директор а есть управляющий директор и кто круче А вот в банках в российских
обычно обычно управляющих круче в западных наоборот часто То есть получается нужно знать
внутреннюю кухню компании абсолютно потому что был у ня один знакомый
работал в юх в Швейцарии и бесился потому что у него написано на визитке
Assistant Vice President Я не ассистент вице-президента я младший вице-президент
То есть это прям было прям очень да то есть там люди на вот эту вот штуку прям сильно Ну получается это даже инструмент
мотивация человека в большой компании то есть там что-нибудь сделаешь и поменяем тебе Ну смотри В структуре сбера до
прихода Дэвида рафаловский
бы вице-президент был старший вице-президент А дальше уже зампред первый зампред и так далее и Греф
собственно А вот для рафаловский [музыка]
там вот эти вот слова типа и перевод этих слов они там вот Окей Какое достижение в своей
карьере ты считаешь самым значимым можешь выбрать несколько если сложно между ними выбрать Слушай что
такое достижение то есть вот как бы я как ты сам определяешь если у тебя такого
Какое достижение в своей карьере Никита считает самым значимым?
поняти Кан State Угу Да И вот вот и и какие у меня дальше возможности наверное
чем больше возможностей я зарезал тем наверное хуже оцениваю Кан сй Вот ну как
так поэтому Ну там достижение Ну это странно то есть ты ты ты же играл в какую-нибудь покер или преф Ну так
совсем чуть-чуть ну те карты сдали ты играешь Ты пытаешься по максимуму играть из того что можно Ну типа взял та два
паровоза на Мизери два вагона Ну сорян бывает это вероятность не так посчитал а
не взял Ну как бы не знаю те подфартило или достижение то есть в некоторых моментах Ну мне кажется можно сказать
что это была удача А в некоторых ну наоборот не повезло То есть ты что-то делаешь максимизирует Как тебе кажется
вот ну там ты же принимаешь решение в какой-то неполных условиях вот потом что-то меняется Вот например мы там
делали наверное сейчас тебе скажу намер месяцев де модель которая предсказывала
цены на недвижку им продаж Ну то есть застройщик строит какой-то объект там
несколько живых корпусов по очереди их вводит в вот а ему надо Ну прогнозировать будущие денежные потоки и
мы эту модель там ну реально там это не одна модель девять моделей там в Каскаде Там ликвидность и так далее мы её
лидировали мы её там обучали там чтобы распределение остатков было определённым образом выглядело чтобы метрики какие-то
там ну там причём там несколько метрик там сразу оптимизировали мы её там обвали год или около того то то есть
пилотировать там Мы там естественно защищали с валидатора то есть всё проверяли всё действительно там датасет
даже собрать для такой задачи по како это же не просто там сайт объявлениями открыть потому что там же не реальная
цена реальная цена которая в сделке Ну то есть их надо как-то вычислить распарсить отчёты застройщиков там целая
наука Вот и потом такое случается Опа А у нас типа двадцать второй февраля
двадцать второго года Ну типа сорян Чему тебя научил Этот проект
Ну что у тебя в любой модели в любом продукте должна быть такая Большая красная кнопка остановиться просто
Отключить всё а о том что надо диверсифицировать никогда там не
[музыка] упаривание не я практически никогда с
плечом не сижу если плечу то там 1-2 про чисто технически там им закрываю вот много вокруг ребят кто там влетел до
сих пор платт брокерам банкам и так далее прям норма так то есть на много лет улетел вот звучит смешно но смешно
когда Тебя это не касается так конечно жалко кстати в недвижке там же ну не только вот это Т история например ты
помнишь когда с 1 января по-моему Эрол счета ввели тоже рынок
поменялся и вот такие достаточно большие
Они конечно да затрудняют историю с моделированием Вот ты так Деликатно обходишь тему достижений это в тебе
скромность или осторожность Окей давай выделю Я когда
перешёл в корпоративный блок тема защиты Фифе тов вот модель она На горизонте только мая непонятно когда я надеюсь на
скорость больших корпораций думал Ну через пару лет дойдут А я уже куда-нибудь на повышение пойду Вот не
успел и кто-то вот из пальца высосал тебе на следующий год надо 10 с по ярдов сделать
типа чистого операционного дохода на моделях А я пришёл ну в чистое поле Там даже витрин Нет там просто вот конь не
валялся какая-то есть отчётность очень-очень хреновая человек который занимался отчётности только через неделю
мне А давай ты все эти три команды отчётности заберёшь Я такой Не чувствуя подвоха типа а зачем
Ну ты же хотел какие-то витрины строить Я ему рассказал какие там Но у меня же ещё кампейн был леды там ки да делай
делай Ну вот Не бывает ничего просто так да а там был швах Да у меня хоп на
следующий год типа защите А там как бы ещё процесс защиты не формализованы А
ещё это сегмент в котором всего на рынке там 100.000 компаний Вот и у него там
распределение дохода банка от этих компаний оно там ну экспоненциально сам понимаешь то есть есть какой-то верхний
квантиль который что-то приносит там нормально денег а хвост он там как бы что-то не делае минимум Копеечка надо
самому придумать инициативу самому продать бизнес а потом в отличие от Ну всяких там розничных ретел истории когда
у тебя там баннерная реклама или какая-то скидка Ты ещё должен убедить клиентского менеджера чтобы он сделал
именно так как рекомендует модель потому что она за него не может принимать решение и делать то есть провести
обучение объяснить почему эта штука вообще работает а когда этих менеджеров полтысячи они по всей стране сидят это
ну не такая простая задача вот договориться о том как это в мотивации будет учитываться Да потому что у них
есть бизнес-план им вообще-то Ну как бы если он много времени будет твои штуки уделять у него он на премию се не
заработает Ну короче это вот был Челлендж А ты даже придумать такие инициативы Потому что ты приходишь
куда-то там типа из разряда Давайте Ну цену на кредит прас такие О так вот уже
два департамента Прат у одних одна модель у других другая у этих на своих данных у этих на своих данными не не
делится потому что другие сразу модели лучше сделают потому что ну как бы фие хочет себе каждый записать ты такой там
типа им объясняешь Ну вот смотрите мы сейчас соберём там ту получается такая внутренняя конкуренция потому что каждый
хочет бонус себе получить не то что бонус получить понимаешь ты вот про позитивную
мотивацию думаешь а там как бы тут про выполнение плана речь идёт да то есть как бы как бы наоборот не получилось вот
и ты такой там хитрыми манипуляциями типа А давайте Пилот проведём Давайте вот так вот это и такой вот в итоге
"""

In [18]:
PROMPT="""
Проанализируй транскрипт интервью.
Определи есть ли ответ на вопрос в этом фрагменте.
Если ответа точного ответа нет, отвечай "ответа нет".
Если ответ есть, сгенерируй ответ исходя из транскрипта.
Отвечай сразу на вопрос, без комментариев и уточнений.

Транскрипт:
{text}


Вопрос:
{question}
"""
import pprint
temperature=0

q="что говрят про получение бонусов?"
resp=llmp(PROMPT.format(text=bigtext,question=q))
pprint.pprint(resp)


MTSAIR/Cotype-Nano,temperature=0,top_p=0.8,max_tokens=2048
('Никита Зелинский отмечает, что в корпоративной среде получение бонусов может '
 'быть связано с двойственностью и бюрократией. Он упоминает, что в крупных '
 'компаниях, таких как МТС, руководители могут получать значительные бонусы за '
 'достижение финансовых показателей, что может привести к несправедливости и '
 'неэффективности. Никита также отмечает, что в некоторых случаях руководители '
 'могут получать значительные бонусы за то, чтобы их менеджеры или сотрудники '
 'продвигались по карьерной лестнице, что может создавать дополнительные '
 'мотивационные проблемы.\n'
 '\n'
 'Он также упоминает, что в корпоративной среде часто используются различные '
 'инициативы, такие как "пилотирование" и "договоры", чтобы мотивировать '
 'сотрудников и стимулировать их на выполнение плана, что может привести к '
 'неэффективной работе и несправедливым распределению бонусов.')
